<a href="https://colab.research.google.com/github/morbosohex/Workflow/blob/master/LSTM_Training%2C_Part_of_Speech_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM for Part-of-Speech Tagging

下面将使用LSTM预测一串词的词性,也即Part-of-Speech（POS）标签

POS标注对一组词中的每个词确定其在上下文中的所处的词性角色，例如 `"noun"`, `"verb"`, `"adjective"`, 等等


### 词性标注的用途
可以用于自然语言处理任务的消歧，帮助确定某人说话过程中的主语，句子生成，语音合成过程中确定正确的读音，信息检索，单词消歧，（例如某个人说的是方向层面的right，还是“that's right"的right

---


### 数据准备
首先，将字符串单词映射为数值，以便之后送入网络

我们选取一些简单的句子，然后将每个句子进行小写处理，然后转变为单词组成的列表，以及对应的每个单词的词性标志（其作为真实的标签）

#### 单词到索引的映射

然后从这个小的训练集中创建词汇字典，其中字典的key是数据集中的单词，value是每个单词对应的唯一索引，然后从数据集的标签部分中，进行同样的操作，将每个标签转变成与之对应的标签：索引的形式

In [0]:
# import resources
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

%matplotlib inline

In [0]:
# training sentences and their corresponding word-tags
training_data = [
    ("The cat ate the cheese".lower().split(), ["DET", "NN", "V", "DET", "NN"]),
    ("She read that book".lower().split(), ["NN", "V", "DET", "NN"]),
    ("The dog loves art".lower().split(), ["DET", "NN", "V", "NN"]),
    ("The elephant answers the phone".lower().split(), ["DET", "NN", "V", "DET", "NN"])
]
        
# create a dictionary that maps words to indices
word2idx = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

# create a dictionary that maps tags to indices
tag2idx = {"DET": 0, "NN": 1, "V": 2}

接下来检查创建的字典，以及对应的数值索引

可以看到每个词都在我们选择的数据集中，而每个索引是唯一的

In [3]:
# print out the created dictionary
print(word2idx)

{'the': 0, 'cat': 1, 'ate': 2, 'cheese': 3, 'she': 4, 'read': 5, 'that': 6, 'book': 7, 'dog': 8, 'loves': 9, 'art': 10, 'elephant': 11, 'answers': 12, 'phone': 13}


In [0]:
import numpy as np

# a helper function for converting a sequence of words to a Tensor of numerical values
# will be used later in training
# 一般步骤，将字符串单词转变为索引，然后将索引序列转变为numpy格式，然后使用pytorch方便的将numpy
# 转变为torch支持的张量

def prepare_sequence(seq, to_idx):
    '''This function takes in a sequence of words and returns a 
    corresponding Tensor of numerical values (indices for each word).'''
    idxs = [to_idx[w] for w in seq]
    idxs = np.array(idxs)
    return torch.from_numpy(idxs)


In [5]:
# check out what prepare_sequence does for one of our training sentences:
example_input = prepare_sequence("The dog answers the phone".lower().split(), word2idx)
print(example_input)

tensor([ 0,  8, 12,  0, 13])


---
## 创建模型
模型的几点假设：
1. 输入是单词序列，类似[w1, w2, ...]
2. 上面的单词来自于一个更大的词汇表，即每个单词都可以在这个词汇表中找到
3. 为了简化问题，只定义了三个标签， `[NN, V, DET]`, which mean: a noun, a verb, and a determinant (words like "the" or "that"), respectively
4. 目标是为每个单词预测一个标签（来自于上述的三个标签）

为了进行预测，我们将会传递给LSTM网络一个测试句子，然后该LSTM网络的末端将会通过一个softmax函数，其中对于每个单词然后返回一个标签打分向量，例如会返回三个值[0.1,0.2, 0.7]，其中最大的值就是该单词所属的标签

数学层面上，可以预测标签$\hat{y}_i$表示为: 

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

其中$A$是学习权重， $b$是学习的偏差， 在`i`时刻的隐藏态是$h_i$. 


### 词向量

我们知道LSTM获取期望的输入size和隐藏层的维度，但是对于一个句子很少有固定的size，那么针对这种情况如何定义LSTM的input


在这个网络的最开始，我们将创建一个“嵌入”层，它接受我们词汇表的大小，并为输入的单词序列中的每个单词返回指定大小的矢量`embedding_dim`。重要的是，这是该网络的第一层。您可以在[PyTorch文档](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#word-embeddings-in-pytorch)中阅读有关此嵌入层的更多信息。
翻

下图是该词性标注的理论架构

![image.png](https://upload-images.jianshu.io/upload_images/12735209-82391e9bd2dbb5ab.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)



In [0]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        ''' Initialize the layers of this model.'''
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim

        # embedding layer that turns words into a vector of a specified size
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # the LSTM takes embedded word vectors (of a specified size) as inputs 
        # and outputs hidden states of size hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # the linear layer that maps the hidden state output dimension 
        # to the number of tags we want as output, tagset_size (in this case this is 3 tags)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        
        # initialize the hidden state (see code below)
        self.hidden = self.init_hidden()

        
    def init_hidden(self):
        ''' At the start of training, we need to initialize a hidden state;
           there will be none because the hidden state is formed based on perviously seen data.
           So, this function defines a hidden state with all zeroes and of a specified size.'''
        # The axes dimensions are (n_layers, batch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        ''' Define the feedforward behavior of the model.'''
        # create embedded word vectors for each word in a sentence
        embeds = self.word_embeddings(sentence)
        
        # get the output and hidden state by passing the lstm over our word embeddings
        # the lstm takes in our embeddings and hiddent state
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        
        # get the scores for the most likely tag for a word
        tag_outputs = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_outputs, dim=1)
        
        return tag_scores


## 定义训练模型

为了训练模型，我们必须实例化该模型并定义我们想要使用的损失和优化器。

首先，我们定义单词嵌入的大小。 `EMBEDDING_DIM`为单词向量的大小;我们将它们保持在较小的位置，以便我们可以看到当我们训练时权重如何变化

**注意：复杂数据集的嵌入维度通常会大得多，大约为64,128或256维。**


#### 损失函数 和 优化器
由于我们的LSTM输出了一系列带有softmax层的标记分数，我们将使用`NLLLoss`。与softmax层一起，NLL Loss创建了我们通常用于分析分数分布的交叉熵损失。我们将使用标准梯度下降优化，但我们鼓励您与其他优化器一起使用！

In [0]:
# the embedding dimension defines the size of our word vectors
# for our simple vocabulary and training set, we will keep these small
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# instantiate our model
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word2idx), len(tag2idx))

# define our loss and optimizer
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


只是为了检查我们的模型是否已经学到了什么，让我们首先看一下*我们的模型训练之前的样本测试句*的分数。请注意，测试句*必须*来自我们词汇表中的单词，否则其单词不能转换为索引。

分数应为长度为3的张量（对应于我们的每个标签），并且输入句子中的每个单词都应该有分数。

对于测试句，“The cheese loves the elephant”，我们知道它有标签（DET，NN，V，DET，NN）或`[0,1,2,0,1]`，但我们的网络确实还不知道这个。事实上，在这种情况下，我们的模型以隐藏的全零状态开始，因此所有分数和预测标记应该是很低的，随机的，因为尚未开始训练

In [10]:
test_sentence = "The cheese loves the elephant".lower().split()

# see what the scores are before training
# element [i,j] of the output is the *score* for tag j for word i.
# to check the initial accuracy of our model, we don't need to train, so we use model.eval()
inputs = prepare_sequence(test_sentence, word2idx)
inputs = inputs
tag_scores = model(inputs)
print(tag_scores)

# tag_scores outputs a vector of tag scores for each word in an inpit sentence
# to get the most likely tag index, we grab the index with the maximum score!
# recall that these numbers correspond to tag2idx = {"DET": 0, "NN": 1, "V": 2}
_, predicted_tags = torch.max(tag_scores, 1)
print('\n')
print('Predicted tags: \n',predicted_tags)

tensor([[-1.3329, -1.3376, -0.7470],
        [-1.2843, -1.3043, -0.7946],
        [-1.2885, -1.3491, -0.7661],
        [-1.3501, -1.3377, -0.7374],
        [-1.2967, -1.3274, -0.7735]], grad_fn=<LogSoftmaxBackward>)


Predicted tags: 
 tensor([2, 2, 2, 2, 2])


---
## 训练模型
loop所有的训练集数据，重复多代（此处为了阐明概念使用较少的epoch），其中loop中包含如下操作：

1. 将模型的梯度归零
2. 初始化隐藏态
3. 准备输入数据
4. 传入输入数据进行前馈操作并返回tag得分
5. 计算tag得分与真实tag之间的损失
6. 反向传播更新权重

在该例子中，每20代打印一次损失

In [11]:
# normally these epochs take a lot longer 
# but with our toy data (only 3 sentences), we can do many epochs in a short time
n_epochs = 300

for epoch in range(n_epochs):
    
    epoch_loss = 0.0
    
    # get all sentences and corresponding tags in the training data
    for sentence, tags in training_data:
        
        # zero the gradients
        model.zero_grad()

        # zero the hidden state of the LSTM, this detaches it from its history
        model.hidden = model.init_hidden()

        # prepare the inputs for processing by out network, 
        # turn all sentences and targets into Tensors of numerical indices
        sentence_in = prepare_sequence(sentence, word2idx)
        targets = prepare_sequence(tags, tag2idx)

        # forward pass to get tag scores
        tag_scores = model(sentence_in)

        # compute the loss, and gradients 
        loss = loss_function(tag_scores, targets)
        epoch_loss += loss.item()
        loss.backward()
        
        # update the model parameters with optimizer.step()
        optimizer.step()
        
    # print out avg loss per 20 epochs
    if(epoch%20 == 19):
        print("Epoch: %d, loss: %1.5f" % (epoch+1, epoch_loss/len(training_data)))


Epoch: 20, loss: 1.01303
Epoch: 40, loss: 0.86895
Epoch: 60, loss: 0.67404
Epoch: 80, loss: 0.53264
Epoch: 100, loss: 0.43052
Epoch: 120, loss: 0.32075
Epoch: 140, loss: 0.22027
Epoch: 160, loss: 0.15084
Epoch: 180, loss: 0.10549
Epoch: 200, loss: 0.07729
Epoch: 220, loss: 0.05939
Epoch: 240, loss: 0.04742
Epoch: 260, loss: 0.03901
Epoch: 280, loss: 0.03285
Epoch: 300, loss: 0.02818


## Testing

See how your model performs *after* training. Compare this output with the scores from before training, above.

Again, for the test sentence, "The cheese loves the elephant", we know that this has the tags (DET, NN, V, DET, NN) or `[0, 1, 2, 0, 1]`. Let's see if our model has learned to find these tags!

In [12]:
test_sentence = "The cheese loves the elephant".lower().split()

# see what the scores are after training
inputs = prepare_sequence(test_sentence, word2idx)
inputs = inputs
tag_scores = model(inputs)
print(tag_scores)

# print the most likely tag index, by grabbing the index with the maximum score!
# recall that these numbers correspond to tag2idx = {"DET": 0, "NN": 1, "V": 2}
_, predicted_tags = torch.max(tag_scores, 1)
print('\n')
print('Predicted tags: \n',predicted_tags)

tensor([[-5.6024e-03, -5.2940e+00, -7.4790e+00],
        [-5.5619e+00, -9.8333e-03, -5.1255e+00],
        [-5.3715e+00, -7.4396e-01, -6.5369e-01],
        [-7.2467e-03, -5.3556e+00, -5.9919e+00],
        [-5.8844e+00, -2.7729e-02, -3.7064e+00]], grad_fn=<LogSoftmaxBackward>)


Predicted tags: 
 tensor([0, 1, 2, 0, 1])


## Great job!

To improve this model, see if you can add sentences to this model and create a more robust speech tagger. Try to initialize the hidden state in a different way or play around with the optimizers and see if you can decrease model loss even faster.